In [241]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

import os

import time

import scipy.stats as stats

import pulp

import unidecode

Loading the data from the files

In [242]:
folderpath = 'data/'

players_1617_df = pd.read_csv(folderpath+'2016-17/players_raw.csv')
players_1718_df = pd.read_csv(folderpath+'2017-18/players_raw.csv')
players_1819_df = pd.read_csv(folderpath+'2018-19/players_raw.csv')
players_1920_df = pd.read_csv(folderpath+'2019-20/players_raw.csv')
players_2021_df = pd.read_csv(folderpath+'2020-21/players_raw.csv')
players_2122_df = pd.read_csv(folderpath+'2021-22/players_raw.csv')
players_2223_df = pd.read_csv(folderpath+'2022-23/players_raw.csv')
players_2324_df = pd.read_csv(folderpath+'2023-24/players_raw.csv')

gws_1617_df = pd.read_csv(folderpath+'2016-17/gws/merged_gw.csv', encoding='latin')
gws_1718_df = pd.read_csv(folderpath+'2017-18/gws/merged_gw.csv', encoding='latin')
gws_1819_df = pd.read_csv(folderpath+'2018-19/gws/merged_gw.csv', encoding='latin')
gws_1920_df = pd.read_csv(folderpath+'2019-20/gws/merged_gw.csv', encoding='latin')
gws_2021_df = pd.read_csv(folderpath+'2020-21/gws/merged_gw.csv', encoding='latin')
gws_2122_df = pd.read_csv(folderpath+'2021-22/gws/merged_gw.csv', encoding='latin')
gws_2223_df = pd.read_csv(folderpath+'2022-23/gws/merged_gw.csv', encoding='latin')
gws_2324_df = pd.read_csv(folderpath+'2023-24/gws/merged_gw.csv', encoding='latin')

team_codes_df = pd.read_csv(folderpath+'teams.csv')
team_codes_df.columns.values[2:] = team_codes_df.columns[2:].str.replace('team_', '')

## Clean and process the dataframes

We want to add:
- Player Position
- Full Name (names are inconsistent across seasons and between df's)

We will also remove `Danny Ward` as there were two in the 18/19 season, both has 0 points so no harm in removing them.

In [243]:
# remove Danny Wards from 18/19 season
players_1819_df = players_1819_df[((players_1819_df.first_name == "Danny") & (players_1819_df.second_name=="Ward"))==False]
gws_1819_df = gws_1819_df[gws_1819_df.name.str.contains("Danny_Ward")==False]

Here we are adding the seasons onto the df

In [244]:
player_df_list = [players_1617_df, players_1718_df, players_1819_df, players_1920_df, players_2021_df, players_2122_df, players_2223_df, players_2324_df]
gw_df_list = [gws_1617_df, gws_1718_df, gws_1819_df, gws_1920_df, gws_2021_df, gws_2122_df, gws_2223_df, gws_2324_df]

# append season and season index to dfs

seasons = ['1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324']
season_nums = list(range(len(seasons)))

for i in range(len(seasons)):
    player_df_list[i]['season'] = seasons[i]
    gw_df_list[i]['season'] = seasons[i]
    
    player_df_list[i]['season_num'] = season_nums[i]
    gw_df_list[i]['season_num'] = season_nums[i]

# combine dataframes from all seasons into one

players_df = pd.concat(player_df_list)
gws_df = pd.concat(gw_df_list)
players_df.reset_index(inplace=True)
gws_df.reset_index(inplace=True)

Then we apply a function to sort out the full names of the players

In [245]:
def get_full_name_playerdf(first_name, second_name):
    '''Creates full name, cleans up accents and makes processing easier'''

    full_name = first_name +'_' + second_name
    full_name = full_name.replace(" ", "_")
    full_name = full_name.replace("-", "_")
    full_name = unidecode.unidecode(full_name)
    
    return full_name

# Translate player positions into string for easier readability
positions_dict = {
    1: 'Keeper',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
    
}

# Now apply full name and position changes. We also add the starting cost by a simple subtraction of two existing data columns.
players_df['full_name'] = players_df.apply(lambda x: get_full_name_playerdf(x.first_name, x.second_name), axis=1).str.lower()
players_df['position'] = players_df.element_type.map(positions_dict)
players_df['starting_cost'] = players_df.now_cost - players_df.cost_change_start_fall
players_df['cost_bin'] = players_df.now_cost.apply(lambda x: np.floor(x/10))

gws_df['full_name'] = gws_df.name.str.replace('_\d+','')
gws_df['full_name'] = gws_df['full_name'].str.replace(" ", "_").str.replace("-", "_").str.replace('_\d+','')
gws_df['full_name'] = gws_df['full_name'].apply(lambda x: unidecode.unidecode(x))
gws_df['full_name'] = gws_df['full_name'].str.lower()

This function gives us a df that contains all the GW info, plus info on the players team names and opponents.

In [246]:
def clean_gw_df(player_df, gw_df, team_codes_df):
    '''
    Cleans and merges gameweek data with player information and team codes to return a DataFrame 
    containing player positions, player's team names, and opponent's team names.

    Returns:
    pd.DataFrame: A DataFrame with the original gameweek data enriched with player positions, 
                player's team names, and opponent's team names.
    '''

    pdf = player_df.copy()[['full_name', 'season', 'position', 'player_team_name']]
    gdf = gw_df.copy()
    gdf = gdf.merge(pdf, on=['full_name', 'season'], how='left')
    
    dfs = []
    for s, group in gdf.groupby('season'):

        temp_code_df = team_codes_df[['team', s]]
        temp_code_df = temp_code_df.dropna()
        
        group = group[['opponent_team']]
        group['opponent_team_name'] = group.opponent_team.map(temp_code_df.set_index(s).team)
        dfs.append(group[['opponent_team_name']])
        
    out_df = pd.concat(dfs, axis=0)
    out_df = pd.concat([gdf, out_df], axis=1)
    return out_df

Finally we apply the above to gather the opponent and player teams to the GW dataframes

In [247]:
gws_df.opponent_team = gws_df.opponent_team.astype(float)
players_df['player_team_name'] = players_df.team_code.map(team_codes_df.set_index('team_code').team)
gws_df = clean_gw_df(players_df, gws_df, team_codes_df)

/var/folders/s7/mksssm8112scb6_3__pw7kb00000gq/T/ipykernel_22838/3669575184.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['opponent_team_name'] = group.opponent_team.map(temp_code_df.set_index(s).team)
/var/folders/s7/mksssm8112scb6_3__pw7kb00000gq/T/ipykernel_22838/3669575184.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['opponent_team_name'] = group.opponent_team.map(temp_code_df.set_index(s).team)
/var/folders/s7/mksssm8112scb6_3__pw7kb00000gq/T/ipykernel_22838/3669575184.py:22:

## Approach

Now we move onto building the best team. There are a variety of rules with FPL with waht team you can build, which will be accounted for. 

We will use a simple model to pick the initial team, and then more complicated ones to decide on transfers etc. We will use the 23-24 season as our test set to test the models.


#### Initial Team

We will look at the latest season (22-23) (as our test set will be 23-24 season results) as the basis for the initial team, optimising fo the maximum number of points scored. As per FPL rules, we have the following rules:

- Total budget of 1000
- 1 keeper
- 4 defenders
- 4 midfielders
- 2 forwards
- No more than 3 players from each team

Bench players are not considered much, and will be picked by the cheapest options.

Let's first get the players are who are available, as some would have been relegated after the previous season (and transferred etc)

In [248]:
def make_available_players_df(this_season_player_df, last_season_player_df):
    """
    Creates a DataFrame of available players for the current season by merging current season 
    player data with their total points from the previous season. If a player did not play 
    last season, their total points are filled with the average total points of players in 
    the same position and cost bin.

    Parameters:
    this_season_player_df (pd.DataFrame): DataFrame containing player data for the current season.
    last_season_player_df (pd.DataFrame): DataFrame containing player data for the previous season, 
                                        including their total points.
    """

    last_season_player_df = last_season_player_df[last_season_player_df.minutes > 0]
    last_season_player_df = last_season_player_df[['full_name', "total_points"]]
    last_season_player_df.rename(columns={'total_points': "total_points_last_season"},
                                inplace=True)
    
    available_players_df = pd.merge(this_season_player_df,
                                    last_season_player_df,
                                    on='full_name', how='left')
    
    available_players_df.total_points_last_season = available_players_df.groupby(['position', 'cost_bin']).total_points_last_season.transform(lambda x: x.fillna(x.mean()))
    
    return available_players_df

# There are two Ben Davies, one plays for Liverpool and got 0 points so removing here, also
# Kane was still in the df for some reason
current_season_player_df = players_df[(players_df.season == '2324') &
                                      ~((players_df.full_name == 'ben_davies') &
                                        (players_df.player_team_name == 'Liverpool'))]

current_season_player_df = players_df[(players_df.season == '2324') &
                                      ~((players_df.full_name == 'harry_kane'))]

previous_season_player_df = players_df[(players_df.season == '2223') &
                                       ~((players_df.full_name == 'ben_davies') &
                                         (players_df.player_team_name == 'Liverpool'))]

available_players_df = make_available_players_df(current_season_player_df, previous_season_player_df)

Now let's get the cheapest players

In [249]:
def get_cheapest_players(player_df):
    """
    Identifies the cheapest players by position and returns their names along with the total cost.

    This function groups the players by their positions, finds the player(s) with the minimum starting 
    cost within each position, and then selects the player with the highest total points among those 
    cheapest players. It collects these players' names and calculates the total cost of these selected players.
    """

    cheapest_player_names = []
    total_cost = 0
    for position, group in player_df.groupby('position'):
        cheapest_players =  group[(group.starting_cost == group.starting_cost.min())]
        top_cheapest_player = cheapest_players[cheapest_players.total_points == cheapest_players.total_points.max()]
        
        cheapest_player_name = top_cheapest_player.full_name.values[0]
        
        cheapest_player_names += [cheapest_player_name]
        total_cost += top_cheapest_player.starting_cost.values[0]
        print(position, ": ", cheapest_player_name )
        
    return cheapest_player_names, total_cost

bench_players, bench_cost = get_cheapest_players(available_players_df)
print(bench_cost)

Defender :  jamaal_lascelles
Forward :  divin_mubama
Keeper :  caoimhin_kelleher
Midfielder :  harrison_reed
139


These are our bench players selected, now let's go for the starting 11. We are going to assume if they did well last season, they will do well this season.

In [250]:
# Define functions with additional checks for NaN values
def make_decision_variables(player_df):
    return [pulp.LpVariable(player, cat="Binary") for player in player_df.full_name]

def make_optimization_function(player_df, decision_variables):
    op_func = pulp.lpSum(
        0 if pd.isna(player_df.total_points_last_season[i]) else player_df.total_points_last_season[i] * decision_variables[i]
        for i in range(len(player_df))
    )
    return op_func

def make_cash_constraint(player_df, decision_variables, available_cash):
    total_paid = pulp.lpSum(
        0 if pd.isna(player_df.starting_cost[i]) else player_df.starting_cost[i] * decision_variables[i]
        for i in range(len(player_df))
    )
    return (total_paid <= available_cash)

def make_player_constraint(position, n, decision_variables, player_df):
    total_n = pulp.lpSum(
        decision_variables[i] for i in range(len(player_df)) if player_df.position[i] == position
    )
    return (total_n == n)

def add_team_constraint(prob, player_df, decision_variables):
    for team, group in player_df.groupby('team_code'):
        team_total = pulp.lpSum(
            decision_variables[i] for i in range(len(player_df)) if player_df.full_name[i] in group.full_name.values
        )
        prob += (team_total <= 3)

available_cash = 1000 - bench_cost

prob = pulp.LpProblem('InitialTeam', pulp.LpMaximize)

decision_variables = make_decision_variables(available_players_df)
prob += make_optimization_function(available_players_df, decision_variables)
prob += make_cash_constraint(available_players_df, decision_variables, available_cash)
prob += make_player_constraint("Keeper", 1, decision_variables, available_players_df) 
prob += make_player_constraint("Defender", 4, decision_variables, available_players_df) 
prob += make_player_constraint("Midfielder", 4, decision_variables, available_players_df) 
prob += make_player_constraint("Forward", 2, decision_variables, available_players_df)

add_team_constraint(prob, available_players_df, decision_variables)

## Solve

prob.writeLP('InitialTeam.lp')
optimization_result = prob.solve()

## Get initial team

def get_initial_team(prob, player_df):
    
    variable_names = [v.name for v in prob.variables()]
    variable_values = [v.varValue for v in prob.variables()]

    initial_team = pd.merge(pd.DataFrame({'full_name': variable_names,
                  'selected': variable_values}),
                                       player_df, on="full_name")
    
    initial_team = initial_team[initial_team.selected==1.0] 
    
    return initial_team

initial_team_df = get_initial_team(prob, available_players_df)
initial_team_df[['full_name', "position", "starting_cost", "player_team_name"]]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/pstanis/Documents/personal_projects/fpl_predictor/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/s7/mksssm8112scb6_3__pw7kb00000gq/T/d3e313c29d2f4010abf55692f5029a48-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/s7/mksssm8112scb6_3__pw7kb00000gq/T/d3e313c29d2f4010abf55692f5029a48-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 5215 RHS
At line 5241 BOUNDS
At line 6106 ENDATA
Problem MODEL has 25 rows, 864 columns and 2592 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2089.41 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 634 columns (634 integ

,full_name,position,starting_cost,player_team_name
92,ben_mee,Defender,44,Brentford
250,erling_haaland,Forward,146,Manchester City
280,gabriel_dos_santos_magalhaes,Defender,58,Arsenal
282,gabriel_martinelli_silva,Midfielder,70,Arsenal
339,ivan_toney,Forward,74,Brentford
485,kieran_trippier,Defender,67,Newcastle United
560,marcus_rashford,Midfielder,78,Manchester United
569,martin_odegaard,Midfielder,87,Arsenal
616,mohamed_salah,Midfielder,143,Liverpool
640,nick_pope,Keeper,51,Newcastle United
